# 6. Tuning

In [ ]:
MODEL = 'XGB'
TUNING_MODEL = 'guangan'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os
import pickle
py_file_location = "./drive/My Drive/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

py_file_location = "./drive/My Drive/LAB/BrownlowH2H/PrivatePackages"
sys.path.append(os.path.abspath(py_file_location))

import NingXiang

import pandas as pd

In [ ]:
ningxiang = NingXiang.NingXiang()

NingXiang Initialised


Read in and Prepare Data

In [ ]:
train_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/train_data.parquet')
val_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/val_data.parquet')
test_data = pd.read_parquet('./drive/My Drive/LAB/BrownlowH2H/data/curated/modelling/test_data.parquet')

In [ ]:
train_other_info = train_data[['game_id', 'player1', 'player2']]
val_other_info = val_data[['game_id', 'player1', 'player2']]
test_other_info = test_data[['game_id', 'player1', 'player2']]

In [ ]:
train_x = train_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
train_y = train_data['Brownlow Votes']
val_x = val_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
val_y = val_data['Brownlow Votes']
test_x = test_data.drop(['game_id', 'player1', 'player2', 'Brownlow Votes'], axis=1)
test_y = test_data['Brownlow Votes']

In [ ]:
# ningxiang.read_in_train_data(train_x, train_y)
# ningxiang.set_model_type("Regression")
# ningxiang_output = ningxiang.get_rf_based_feature_combinations(3)

# with open(f'./drive/My Drive/LAB/BrownlowH2H/models/NingXing_output.pickle', 'rb') as f:
#     ningxiang_output = pickle.load(f)

In [ ]:
with open('./drive/My Drive/LAB/BrownlowH2H/models/H2H_NingXiang.pickle', 'rb') as f:
  ningxiang_output = pickle.load(f)

Set parameters

In [ ]:
import GuangAn_H2H

In [ ]:
guangan = GuangAn_H2H.GuangAn()

GuangAn Initialised


In [ ]:
from xgboost import XGBRegressor as XGB

In [ ]:
guangan.read_in_model(XGB, 'Regression')

Successfully read in model <class 'xgboost.sklearn.XGBRegressor'>, which is a Regression model


In [ ]:
guangan.read_in_data(train_x, train_y, val_x, val_y, test_x, test_y)

Read in Train X data
Read in Train y data
Read in Val X data
Read in Val y data
Read in Test X data
Read in Test y data


In [ ]:
parameter_choices = {
    'n_estimators': set(range(25, 201)),
    'max_depth': set(range(3, 97)),
    'subsample': (0.4, 0.85),
    'colsample_bytree': (0.4, 0.85),
    'gamma': (-8, -1),
    'eta': (-4, 1)
}

In [ ]:
guangan.set_hyperparameters(parameter_choices)

Successfully recorded hyperparameter choices


In [ ]:
guangan.read_in_transform({'eta': '10^', 'gamma': '10^'})

Updated transform dictionary: {'n_estimators': False, 'max_depth': False, 'subsample': False, 'colsample_bytree': False, 'gamma': '10^', 'eta': '10^'}


In [ ]:
guangan.set_non_tuneable_hyperparameters({'random_state': 19260817})

Successfully recorded non_tuneable_hyperparameter choices


In [ ]:
guangan.set_features(ningxiang_output)

Successfully recorded tuneable feature combination choices and updated relevant internal structures


In [ ]:
try:
  guangan.read_in_tuning_result_df(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv', f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')
  print('True')
except:
  print('False')

Successfully read in tuning result of 3050 rows
True


In [ ]:
guangan.set_tuning_result_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tuning/{TUNING_MODEL}_BrownlowH2H_{MODEL}.csv')

Successfully set tuning output address


In [ ]:
guangan.set_best_model_saving_address(f'./drive/My Drive/LAB/BrownlowH2H/models/tmp_models/{TUNING_MODEL}_BrownlowH2H_{MODEL}.pickle')

Successfully set best model output address


In [ ]:
guangan.train_other_info = train_other_info
guangan.val_other_info = val_other_info
guangan.test_other_info = test_other_info

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
guangan.tune()

Streaming output truncated to the last 5000 lines.
Already Trained and Tested combination {'n_estimators': 25, 'max_depth': 50, 'subsample': 0.625, 'colsample_bytree': 0.625, 'gamma': -8, 'eta': -1.5, 'features': 35}, which had val score of 0.5723
        Current best combo: (113, 50, 0.85, 0.85, -1, -1.5, 35) with val score 0.6237. 
        Has trained 3050 combinations so far
Already Trained and Tested combination {'n_estimators': 25, 'max_depth': 50, 'subsample': 0.625, 'colsample_bytree': 0.625, 'gamma': -4.5, 'eta': -4, 'features': 18}, which had val score of 0.2117
        Current best combo: (113, 50, 0.85, 0.85, -1, -1.5, 35) with val score 0.6237. 
        Has trained 3050 combinations so far
Already Trained and Tested combination {'n_estimators': 25, 'max_depth': 50, 'subsample': 0.625, 'colsample_bytree': 0.625, 'gamma': -4.5, 'eta': -4, 'features': 35}, which had val score of 0.2117
        Current best combo: (113, 50, 0.85, 0.85, -1, -1.5, 35) with val score 0.6237. 
    

In [ ]:
guangan.view_best_combo_and_score()